# How to access Google APIs

The goal is to learn about google APIs and understand how to access and interact with them.

In [1]:
# extension to help clean python code
%load_ext lab_black

## 1. Packages to install

In [2]:
import json
import requests
import yaml

## 2. Get Google APIs credentials

Login via Google to [Google Maps APIs](https://console.cloud.google.com/google/maps-apis/home), go through the process and get your key.

In [ ]:
# store key in dictionary
key_dict = {"key": "yourkey"}

In [4]:
# dump key into yaml
with open("google_credentials.yml", "w") as output:
    yaml.dump(key_dict, output)

In [5]:
# get key from .yaml file and create gmaps_key variable
key_dict = yaml.load(open("google_credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]

In [ ]:
# display key
gmaps_key

Important: remember to add the .yml file in the .gitignore.

## 3. Send a query to Google Maps API

[Text search](https://developers.google.com/maps/documentation/places/web-service/search-text) is a query that helps us in finding information about a set of places based on a string. The API responds with a list of places matching the text string and any location bias that has been set.

In [ ]:
# a text search request is an HTTP URL of the following form
https://maps.googleapis.com/maps/api/place/textsearch/output?parameters

According to the text search request walktrough on Google, the output can be either a JSON or an XML.

In [8]:
# url to use for query
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

In [11]:
# text input string on which to search
query = input("Search query: ")

# api key to use
api_key = gmaps_key

# get method of requests module to return response object
req = requests.get(url + "query=" + query + "&key=" + api_key)

# convert json format data into python format data (deserialize)
places_json = req.json()

# store results -> list of nested dictionaries
my_result = places_json["results"]

Search query:  restaurants zürich


In [12]:
# check first element
my_result[0]

{'business_status': 'OPERATIONAL',
 'formatted_address': 'Schlüsselgasse 8, 8001 Zürich, Switzerland',
 'geometry': {'location': {'lat': 47.37096469999999, 'lng': 8.5411972},
  'viewport': {'northeast': {'lat': 47.37227747989272,
    'lng': 8.542589979892721},
   'southwest': {'lat': 47.36957782010727, 'lng': 8.539890320107277}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'name': 'Veltlinerkeller',
 'opening_hours': {'open_now': False},
 'photos': [{'height': 2592,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103263139868840012716">Pitsch Patsch</a>'],
   'photo_reference': 'AfLeUgM1wjT2as07lCGoiFaC9nJUD0yoeMXERhzmQfOPgSn43k8GieqVb0UPLOhLHb3lQwkvvhSMUOeQUE31bb3e8ufCPNa9EHuOqaNkM-05Av3klH1R6DcoHOGczT56CwfDUrMugAjRJjzlWnepSOuAdrDV_itAAFzThMILmMjGatPuR-vB',
   'width': 4608}],
 'pla

In [13]:
# number of results from query
len(my_result)

20

In [16]:
# loop through result items to get all restaurant names
for item in my_result:
    # print 'name' key
    print(item["name"])

Veltlinerkeller
Thach
Madrid
Pavillon
Zunfthaus zur Waag
Zeughauskeller
Weisses Rössli
Drei Stuben
Restaurant Kindli
Tao‘s
Lindenhofkeller Restaurant
Josef
Carlton
Osso
Heugümper
Differente
Eichhörnli
Aurora
Anna
The Restaurant


In [18]:
# inspect 'location' key (GeoJSON)
my_result[0]["geometry"]

{'location': {'lat': 47.37096469999999, 'lng': 8.5411972},
 'viewport': {'northeast': {'lat': 47.37227747989272,
   'lng': 8.542589979892721},
  'southwest': {'lat': 47.36957782010727, 'lng': 8.539890320107277}}}

In [21]:
# restaurant names, addresses and ratings from deserialized my_result
def my_result_filtered():
    for item in my_result:
        print(item["name"], item["formatted_address"], item["rating"])


my_result_filtered()

Veltlinerkeller Schlüsselgasse 8, 8001 Zürich, Switzerland 4.7
Thach Militärstrasse 84, 8004 Zürich, Switzerland 4.6
Madrid Froschaugasse 15, 8001 Zürich, Switzerland 4.4
Pavillon Talstrasse 1, 8001 Zürich, Switzerland 4.7
Zunfthaus zur Waag Münsterhof 8, 8001 Zürich, Switzerland 4.6
Zeughauskeller Bahnhofstrasse 28A, 8001 Zürich, Switzerland 4.4
Weisses Rössli Bederstrasse 96, 8002 Zürich, Switzerland 4.8
Drei Stuben Beckenhofstrasse 5, 8006 Zürich, Switzerland 4.5
Restaurant Kindli Strehlgasse 24, 8001 Zürich, Switzerland 4.7
Tao‘s Augustinergasse 3, 8001 Zürich, Switzerland 4.2
Lindenhofkeller Restaurant Pfalzgasse 4, 8001 Zürich, Switzerland 4.7
Josef Gasometerstrasse 24, 8005 Zürich, Switzerland 4.4
Carlton Bahnhofstrasse 41, 8001 Zürich, Switzerland 4.3
Osso Zollstrasse 121, 8005 Zürich, Switzerland 4.2
Heugümper Waaggasse 4, 8001 Zürich, Switzerland 4.7
Differente Schaffhauserstrasse 1, 8006 Zürich, Switzerland 4.6
Eichhörnli Nietengasse 16, 8004 Zürich, Switzerland 4.7
Aurora B

In [22]:
# normalize data via json_normalize into a flat table (pandas)
import pandas as pd

pd.json_normalize(my_result)

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,OPERATIONAL,"Schlüsselgasse 8, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Veltlinerkeller,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJO1F5rwAKkEcRJ7H4nRupWr4,3.0,4.7,...,206,47.370965,8.541197,47.372277,8.542590,47.369578,8.539890,False,9GCR+9F Zürich,8FVC9GCR+9F
1,OPERATIONAL,"Militärstrasse 84, 8004 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Thach,"[{'height': 1078, 'html_attributions': ['<a hr...",ChIJqZ09QxAKkEcRKP3hQwz-9Rc,1.0,4.6,...,661,47.378669,8.529902,47.379981,8.531216,47.377281,8.528517,True,9GHH+FX Zürich,8FVC9GHH+FX
2,OPERATIONAL,"Froschaugasse 15, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Madrid,"[{'height': 9248, 'html_attributions': ['<a hr...",ChIJVyhQpamgmkcRbttRg83N9a0,2.0,4.4,...,524,47.373241,8.544660,47.374616,8.546020,47.371917,8.543320,True,9GFV+7V Zürich,8FVC9GFV+7V
3,OPERATIONAL,"Talstrasse 1, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Pavillon,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJnyo69v8JkEcR5m9QvswpaRo,4.0,4.7,...,156,47.367175,8.539297,47.368584,8.540795,47.365885,8.538095,False,9G8Q+VP Zürich,8FVC9G8Q+VP
4,OPERATIONAL,"Münsterhof 8, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Zunfthaus zur Waag,"[{'height': 4025, 'html_attributions': ['<a hr...",ChIJi--VlgAKkEcRUkK9UpnXcdI,3.0,4.6,...,762,47.370219,8.540326,47.371526,8.541673,47.368826,8.538973,True,9GCR+34 Zürich,8FVC9GCR+34
5,OPERATIONAL,"Bahnhofstrasse 28A, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Zeughauskeller,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJX7fB6wAKkEcRhpTedk8sztg,2.0,4.4,...,8926,47.370421,8.539974,47.371733,8.541242,47.369033,8.538542,True,9GCQ+5X Zürich,8FVC9GCQ+5X
6,OPERATIONAL,"Bederstrasse 96, 8002 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Weisses Rössli,"[{'height': 683, 'html_attributions': ['<a hre...",ChIJ77wVo_AJkEcR9PPGktVXLgA,3.0,4.8,...,606,47.361912,8.526144,47.363235,8.527559,47.360535,8.524859,True,9G6G+QF Zürich,8FVC9G6G+QF
7,OPERATIONAL,"Beckenhofstrasse 5, 8006 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Drei Stuben,"[{'height': 5522, 'html_attributions': ['<a hr...",ChIJMRcesAsKkEcRYEUf90e-Bsc,3.0,4.5,...,353,47.383272,8.541610,47.384610,8.543027,47.381911,8.540327,True,9GMR+8J Zürich,8FVC9GMR+8J
8,OPERATIONAL,"Strehlgasse 24, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Restaurant Kindli,"[{'height': 681, 'html_attributions': ['<a hre...",ChIJF45_RwcKkEcR4PZSHMC1Fmg,NaN,4.7,...,326,47.372335,8.540653,47.373636,8.541984,47.370936,8.539284,True,9GCR+W7 Zürich,8FVC9GCR+W7
9,OPERATIONAL,"Augustinergasse 3, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Tao‘s,"[{'height': 2700, 'html_attributions': ['<a hr...",ChIJL3iOygAKkEcRS543QQANWGU,3.0,4.2,...,1012,47.371448,8.540199,47.372846,8.541594,47.370146,8.538895,True,